## Welcome to You.com's Search API
### Access Keys
You will need to have the environment variables `AUTH_KEY` and `COHERE_API_KEY` set for this notebook to fully work.

In [ ]:
!pip install cohere

In [ ]:
import os


if not os.getenv("AUTH_KEY") or not os.getenv("COHERE_API_KEY"):
    raise RuntimeError("You need to set both AUTH_KEY and COHERE_API_KEY environment variables to proceed")

    
try:
    import cohere
except:
    raise RuntimeError("Cohere library is not installed")

    
cohere_client = cohere.Client(os.getenv("COHERE_API_KEY"))

### Define Utility Functions
We use the requests library to interface with the You.com API currently with plans to have a dedicated Python package in the near future. We use the Cohere Python client to interface with the Command model.

In [ ]:
import requests

from itertools import chain


def get_ai_snippets_for_query(query):
    headers = {"X-API-Key": os.environ["AUTH_KEY"]}
    results = requests.get(
        f"https://api.ydc-index.io/search?query={query}",
        headers=headers,
    ).json()
        
    # We return many text snippets for each search hit so we need to explode both levels
    return "\n".join(["\n".join(hit["snippets"]) for hit in results["hits"]])


def get_cohere_prompt(query, context):
    return f"""given a question and a bunch of snippets context try to answer the question using the context. If you can't please say 'Sorry hooman, no dice'.
question: {query}
context: {context}
answer: """


def ask_cohere(query, context):
    try:
        return cohere_client.generate(prompt=get_cohere_prompt(query, context))[
            0
        ].text
    except:
        logging.error(
            f"Cohere call failed for query {query} and context {context}",
            exc_info=True,
        )
        return "Sorry hooman, no dice"


def ask_cohere_with_ai_snippets(query):
    ai_snippets = get_ai_snippets_for_query(query)
    return ask_cohere(query, ai_snippets)

### Now ask a question!

In [ ]:
ask_cohere_with_ai_snippets("Who is the CEO of Cohere?")